In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import plotly.express as px
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [78]:
df = pd.read_csv("retouren.csv")
df["user_dob"] = pd.to_datetime(df.user_dob)
df["user_reg_date"] = pd.to_datetime(df.user_reg_date)
df["order_date"] = pd.to_datetime(df.order_date)
df["delivery_date"] = pd.to_datetime(df.delivery_date)
df = df.drop(df[df.item_price == 999].index)
df = df.drop(df[df.delivery_date < df.order_date].index)
df = df.drop(df[df.item_color == "?"].index)
df.loc[df.item_color == "brwon","item_color"] = "brown"
df.loc[df.item_color == "blau","item_color"] = "blue"
df.loc[df.item_color == "dark oliv","item_color"] = "dark olive"
df["age"] = df.order_date.dt.year - df.user_dob.dt.year -((df.order_date.dt.date)<(df.user_dob.dt.date))
df["acc_age"] = (df.order_date - df.user_reg_date).dt.days
#df["acc_age"] = df.acc_age.dt.days
df_mark = df.groupby(["item_id","item_size","item_color"]).agg(maxprice=("item_price","max")).reset_index()
df = pd.merge(df,df_mark)
df["markdown"] = 0
df.loc[df.item_price < df.maxprice,"markdown"] = 1
df.drop(df[df.delivery_date.isna()].index, inplace=True)

df.loc[df.age > 100, "age"] = df.loc[df.age > 100, "age"] - 100
bins=[0,10,20,30,40,50,60,70,80,90,100]
df["age"] = pd.cut(df.age,bins=bins)

df.sort_values('order_date', inplace=True)

# Generate order ID and order size
order_id_generator = df.groupby(['order_date', 'user_id'], sort= False).size().reset_index(name='order_size')
order_id_generator['order_id'] = order_id_generator.index

df = pd.merge(df,order_id_generator,on=['user_id', 'order_date'], how='left')

# Generate the number of the order per user
df_nth_order_of_user = df.groupby(['user_id', 'order_id'], as_index=False).size().drop('size', axis=1)
df_nth_order_of_user['nth_order_of_user'] = df_nth_order_of_user.groupby(['user_id']).cumcount()+1

df = pd.merge(df,df_nth_order_of_user,on=['user_id', 'order_id'], how='left')

# Generate if item id appears multiple times in one order
item_counts_in_order = df.groupby(['order_id', 'item_id'], as_index=False).size()
item_counts_in_order['item_id_>1_in_order'] = item_counts_in_order['size'] > 1

df = pd.merge(df,item_counts_in_order[['order_id', 'item_id', 'item_id_>1_in_order']],on=['order_id', 'item_id'], how='left')

# Generate if item id appears in previous order
df['item_id_ordered_previously'] = False

for index, row in df.iterrows():
    item_id = row['item_id']
    order_id = row['order_id']
    user_id = row['user_id']
    if user_id == 9: print(user_id, order_id, item_id)
    for i2, prev_row in df[df.user_id == user_id].iterrows():
        if user_id == 9: print(f"{'':15}{prev_row['item_id']}")
        if prev_row['order_id'] == order_id: 
            break
        if prev_row['item_id'] == item_id: 
            df.loc[index, 'item_id_ordered_previously'] = True
            break
        

#print(df.loc[df.order_id == 0])
print(df.head())
df_reduced = df[df.user_id < 100].sort_values('order_date')
df_reduced.groupby('user_id').apply(display)


9 12605 1922
               1922
9 12605 1613
               1922
9 12605 2081
               1922
9 16463 1613
               1922
               1613
9 16463 2081
               1922
               1613
               2081
   order_item_id order_date delivery_date  item_id item_size item_color  \
0              1 2016-06-22    2016-06-27      643        38       navy   
1            641 2016-06-22    2016-06-28      100        39        red   
2            620 2016-06-22    2016-07-04      523        xl    magenta   
3            621 2016-06-22    2016-07-04      523        xl    magenta   
4            548 2016-06-22    2016-06-26      173         m      brown   

   brand_id  item_price  user_id user_title  ... return       age acc_age  \
0        30        49.9    30822        Mrs  ...      0  (40, 50]      -1   
1        43        69.9    30931         Mr  ...      1  (40, 50]     491   
2         3         7.9    30929        Mrs  ...      1  (50, 60]     491   
3         3     

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,return,age,acc_age,maxprice,markdown,order_size,order_id,nth_order_of_user,item_id_>1_in_order,item_id_ordered_previously
40006,47261,2016-08-01,2016-08-24,1613,40,ancient,1,199.9,9,Mrs,...,1,"(40, 50]",152,199.9,0,3,12605,1,False,False
40401,47259,2016-08-01,2016-08-02,2081,40,black,40,219.9,9,Mrs,...,1,"(40, 50]",152,219.9,0,3,12605,1,False,False
40003,47260,2016-08-01,2016-08-02,1922,40,grey,26,159.9,9,Mrs,...,1,"(40, 50]",152,159.9,0,3,12605,1,False,False
54918,62930,2016-08-10,2016-08-14,1613,40,black,1,199.9,9,Mrs,...,0,"(40, 50]",161,199.9,0,2,16463,2,False,True
55001,62931,2016-08-10,2016-08-14,2081,40,black,40,219.9,9,Mrs,...,1,"(40, 50]",161,219.9,0,2,16463,2,False,True


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,return,age,acc_age,maxprice,markdown,order_size,order_id,nth_order_of_user,item_id_>1_in_order,item_id_ordered_previously
19324,24204,2016-07-02,2016-07-03,966,37,turquoise,69,44.9,11,Mrs,...,0,"(40, 50]",501,44.9,0,1,5589,1,False,False


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,return,age,acc_age,maxprice,markdown,order_size,order_id,nth_order_of_user,item_id_>1_in_order,item_id_ordered_previously
20717,26128,2016-07-03,2016-07-04,103,41,grey,17,69.9,24,Mrs,...,0,"(50, 60]",121,99.9,1,1,5962,1,False,False


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,return,age,acc_age,maxprice,markdown,order_size,order_id,nth_order_of_user,item_id_>1_in_order,item_id_ordered_previously
59448,67312,2016-08-13,2016-08-15,1658,m,black,90,189.9,30,Mrs,...,0,"(60, 70]",204,189.9,0,2,17326,1,True,False
58472,67311,2016-08-13,2016-08-15,1658,l,black,90,189.9,30,Mrs,...,1,"(60, 70]",204,189.9,0,2,17326,1,True,False


...

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,return,age,acc_age,maxprice,markdown,order_size,order_id,nth_order_of_user,item_id_>1_in_order,item_id_ordered_previously
17346,21201,2016-07-01,2016-07-23,557,38,pink,1,64.9,90,Mrs,...,0,"(50, 60]",500,64.9,0,1,5009,1,False,False


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,return,age,acc_age,maxprice,markdown,order_size,order_id,nth_order_of_user,item_id_>1_in_order,item_id_ordered_previously
10697,12570,2016-06-28,2016-07-02,415,41,green,1,44.9,94,Mrs,...,1,"(50, 60]",120,69.9,1,4,2829,1,False,False
10696,12569,2016-06-28,2016-07-02,334,40,grey,72,49.9,94,Mrs,...,0,"(50, 60]",120,49.9,0,4,2829,1,False,False
9302,12572,2016-06-28,2016-07-05,860,l,ocher,42,49.9,94,Mrs,...,0,"(50, 60]",120,99.9,1,4,2829,1,False,False
10689,12571,2016-06-28,2016-07-02,119,41,blue,43,44.9,94,Mrs,...,1,"(50, 60]",120,59.9,1,4,2829,1,False,False
16493,20390,2016-06-30,2016-07-24,32,m,red,3,12.4,94,Mrs,...,0,"(50, 60]",122,24.9,1,4,4640,2,False,False
16180,20392,2016-06-30,2016-07-27,370,l,ocher,31,44.9,94,Mrs,...,0,"(50, 60]",122,49.9,1,4,4640,2,False,False
15792,20389,2016-06-30,2016-07-24,58,l,green,31,44.9,94,Mrs,...,0,"(50, 60]",122,44.9,0,4,4640,2,False,False
16042,20391,2016-06-30,2016-07-24,434,l,green,28,49.9,94,Mrs,...,0,"(50, 60]",122,59.9,1,4,4640,2,False,False
31836,38667,2016-07-20,2016-07-23,1409,m,petrol,3,21.9,94,Mrs,...,0,"(50, 60]",142,21.9,0,8,9880,3,True,False
31848,38666,2016-07-20,2016-07-23,1628,l,petrol,3,24.9,94,Mrs,...,0,"(50, 60]",142,24.9,0,8,9880,3,True,False


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,return,age,acc_age,maxprice,markdown,order_size,order_id,nth_order_of_user,item_id_>1_in_order,item_id_ordered_previously
29043,33973,2016-07-13,2016-07-16,367,43,blue,18,99.90,98,Mrs,...,1,"(10, 20]",512,99.90,0,5,8936,1,False,False
28995,33974,2016-07-13,2016-07-16,375,44,brown,13,49.90,98,Mrs,...,0,"(10, 20]",512,49.90,0,5,8936,1,False,False
28863,33971,2016-07-13,2016-07-16,1373,34,red,69,39.90,98,Mrs,...,0,"(10, 20]",512,39.90,0,5,8936,1,False,False
28847,33970,2016-07-13,2016-07-16,1333,33,purple,69,42.90,98,Mrs,...,1,"(10, 20]",512,42.90,0,5,8936,1,False,False
28761,33972,2016-07-13,2016-07-16,1332,35,red,69,44.90,98,Mrs,...,1,"(10, 20]",512,44.90,0,5,8936,1,False,False
59359,67260,2016-08-13,2016-08-14,1402,33,red,69,99.95,98,Mrs,...,1,"(10, 20]",543,99.95,0,15,17369,2,True,False
59723,67267,2016-08-13,2016-08-20,2024,27,blue,51,69.90,98,Mrs,...,1,"(10, 20]",543,69.90,0,15,17369,2,True,False
59472,67257,2016-08-13,2016-08-14,1402,30,red,69,99.95,98,Mrs,...,1,"(10, 20]",543,99.95,0,15,17369,2,True,False
59375,67269,2016-08-13,2016-08-20,2024,29,blue,51,69.90,98,Mrs,...,1,"(10, 20]",543,69.90,0,15,17369,2,True,False
59371,67268,2016-08-13,2016-08-20,2024,28,blue,51,69.90,98,Mrs,...,1,"(10, 20]",543,69.90,0,15,17369,2,True,False


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,return,age,acc_age,maxprice,markdown,order_size,order_id,nth_order_of_user,item_id_>1_in_order,item_id_ordered_previously
2301,2836,2016-06-25,2016-06-26,128,l,ash,49,49.90,99,Mrs,...,0,"(40, 50]",86,49.90,0,3,776,1,False,False
2306,2837,2016-06-25,2016-06-26,502,10,green,21,45.90,99,Mrs,...,0,"(40, 50]",86,45.90,0,3,776,1,False,False
2573,2838,2016-06-25,2016-07-18,171,xl,black,3,24.90,99,Mrs,...,0,"(40, 50]",86,24.90,0,3,776,1,False,False
4484,9002,2016-06-27,2016-07-02,465,l,grey,49,29.90,99,Mrs,...,0,"(40, 50]",88,59.90,1,5,1526,2,True,False
4685,9001,2016-06-27,2016-07-02,465,l,ash,49,29.90,99,Mrs,...,0,"(40, 50]",88,59.90,1,5,1526,2,True,False
5928,8999,2016-06-27,2016-07-02,128,l,grey,49,39.90,99,Mrs,...,0,"(40, 50]",88,79.90,1,5,1526,2,True,True
7176,9000,2016-06-27,2016-07-02,128,m,ash,49,39.90,99,Mrs,...,0,"(40, 50]",88,39.90,0,5,1526,2,True,True
7177,8998,2016-06-27,2016-07-02,263,m,green,5,34.90,99,Mrs,...,0,"(40, 50]",88,39.90,1,5,1526,2,False,False
50629,57348,2016-08-07,2016-08-09,2071,unsized,red,48,16.95,99,Mrs,...,0,"(40, 50]",129,16.95,0,5,15166,3,True,False
50398,57350,2016-08-07,2016-08-10,1517,42,ocher,6,69.90,99,Mrs,...,0,"(40, 50]",129,69.90,0,5,15166,3,False,False


""


In [72]:
df.groupby('item_id_ordered_previously').size()

item_id_ordered_previously
False    66110
dtype: int64

In [5]:
def print_evaluation(pipeline_or_model, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names):
    """ Ausgabe von R2-Wert, MSE und MAE für Trainings- und Testset """
    # r2_train = r2_score(y_train, y_train_pred)
    # mse_train = mean_squared_error(y_train, y_train_pred)
    # mae_train = mean_absolute_error(y_train, y_train_pred)

    accurary_train = accuracy_score(y_train, y_train_pred)
    recall_train = recall_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred)

    # r2_test = r2_score(y_test, y_test_pred)
    # mse_test = mean_squared_error(y_test, y_test_pred)
    # mae_test = mean_absolute_error(y_test, y_test_pred)

    accurary_test = accuracy_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    
    print(
        f"{pipeline_or_model} Evaluation:\n"
        f"{'':6} {'ACC':>10} | {'REC':>14} | {'PRE':>10} | {'rows':>8} | {'columns':>8}\n"
        f"{'Train':6} {accurary_train:10.5f} | {recall_train:14.5f} | {precision_train:10.5f} | {X_train.shape[0]:8} | {X_train.shape[1]:8}\n"
        f"{'Test':6} {accurary_test:10.5f} | {recall_test:14.5f} | {precision_test:10.5f} | {X_test.shape[0]:8} | {X_test.shape[1]:8}\n"
    )

In [80]:
X = pd.get_dummies(df,columns=["user_title","user_state","markdown","age",'item_color', 'item_size'],dummy_na=True)
y = df['return']
#X = pd.get_dummies(df[["item_id", "item_color",	"brand_id",	"item_price",	"user_id",	"user_title",	"user_state",	"markdown", "age","acc_age", "item_size" ]])
X.drop(columns=['order_item_id','order_date', 'delivery_date', 'user_dob', 'user_reg_date', 'return', 'order_id'], inplace=True)
X

,item_id,brand_id,item_price,user_id,acc_age,maxprice,order_size,nth_order_of_user,item_id_>1_in_order,item_id_ordered_previously,...,item_size_90,item_size_l,item_size_m,item_size_s,item_size_unsized,item_size_xl,item_size_xs,item_size_xxl,item_size_xxxl,item_size_nan
0,643,30,49.90,30822,-1,49.90,1,1,False,False,...,0,0,0,0,0,0,0,0,0,0
1,100,43,69.90,30931,491,69.90,12,1,True,False,...,0,0,0,0,0,0,0,0,0,0
2,523,3,7.90,30929,491,12.90,10,1,True,False,...,0,0,0,0,0,1,0,0,0,0
3,523,3,7.90,30929,491,12.90,10,1,True,False,...,0,0,0,0,0,1,0,0,0,0
4,173,20,19.90,18912,491,49.90,8,1,False,False,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66105,1749,43,99.90,4466,550,99.90,6,3,False,False,...,0,0,0,0,0,0,0,0,0,0
66106,1485,42,149.90,30564,59,149.90,18,2,True,False,...,0,1,0,0,0,0,0,0,0,0
66107,1470,3,39.90,43724,550,39.90,10,1,False,False,...,0,1,0,0,0,0,0,0,0,0
66108,608,15,32.95,36953,35,32.95,2,2,False,True,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
##### WITH SIZES

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf = RandomForestClassifier()
rf.fit(X=X_train, y=y_train)


parameters = {"n_estimators": [100, 150, 200, 300], "max_depth": [20, 30, 40], 'max_features': [25, 40, 50], 'bootstrap': [True, False], 'oob_score': [False, True], 'ccp_alpha': [0, 0.005, 0.01], 'criterion': ['gini', 'entropy']}
rf_gscv = GridSearchCV(rf, parameters, cv=5,n_jobs=-1,verbose=3)
rf_gscv.fit(X_train, y_train)

y_train_pred_cv = rf_gscv.predict(X_train)
y_test_pred_cv = rf_gscv.predict(X_test)
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)


In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf = RandomForestClassifier(ccp_alpha = 0, n_estimators=400, criterion='entropy', max_depth=30, max_features=35)
rf.fit(X=X_train, y=y_train)

y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print("-"*100)
print_evaluation(rf, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names=X.columns)
print("-"*100)

----------------------------------------------------------------------------------------------------
RandomForestClassifier(ccp_alpha=0, criterion='entropy', max_depth=30,
                       max_features=35, n_estimators=400) Evaluation:
              ACC |            REC |        PRE |     rows |  columns
Train     0.99663 |        0.99869 |    0.99468 |    52888 |      220
Test      0.63992 |        0.65252 |    0.63877 |    13222 |      220

----------------------------------------------------------------------------------------------------


In [68]:
### WITHOUT SIZES

X = pd.get_dummies(df,columns=["user_title","user_state","markdown","age",'item_color'],dummy_na=True)
y = df['return']
X.drop(columns=['order_item_id','order_date', 'delivery_date', 'user_dob', 'user_reg_date', 'return', 'item_size'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rf = RandomForestClassifier()
rf.fit(X=X_train, y=y_train)

# parameters = {"max_depth": [10, 20, 30, 50], 'max_features': ["sqrt",25, 40, 50], 'bootstrap': [True, False], 'oob_score': [False, True], 'ccp_alpha': [0, 0.005, 0.01]}
# parameters = {"max_depth": [20, 50], 'max_features': ["sqrt",25, 40, 50],'ccp_alpha': [0, 0.005, 0.01]}
rf_gscv = GridSearchCV(rf, parameters, cv=5,n_jobs=-1,verbose=3)
rf_gscv.fit(X_train, y_train)

y_train_pred_cv = rf_gscv.predict(X_train)
y_test_pred_cv = rf_gscv.predict(X_test)
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

print("-"*100)
print(rf_gscv.best_estimator_)
print_evaluation(rf_gscv, X_train, X_test, y_train, y_test, y_train_pred_cv, y_test_pred_cv, feature_names=X.columns)
print("-"*100)
print(rf.get_params())
print_evaluation(rf, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred, feature_names=X.columns)
print(rf_gscv.cv_results_['mean_test_score'])
print(rf_gscv.cv_results_['params'])

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1080 fits failed out of a total of 4320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1080 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_forest.py", line 437, in fit
    raise ValueError("Out of bag estimation only available if bootstrap=True")
ValueError: Out of bag estimation onl

----------------------------------------------------------------------------------------------------
RandomForestClassifier(ccp_alpha=0, criterion='entropy', max_depth=20,
                       max_features=50, n_estimators=150)
GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'ccp_alpha': [0, 0.005, 0.01],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [20, 30, 40],
                         'max_features': [25, 40, 50],
                         'n_estimators': [100, 150, 200, 300],
                         'oob_score': [False, True]},
             verbose=3) Evaluation:
              ACC |            REC |        PRE |     rows |  columns
Train     0.95619 |        0.97642 |    0.93926 |    52888 |      119
Test      0.61443 |        0.68439 |    0.60552 |    13222 |      119

-----------------------------------------------------------------

In [34]:
order_id_generator = df.groupby(['user_id', 'order_date']).size().reset_index(name='order_size')
order_id_generator['order_id'] = order_id_generator.index
order_id_generator

,user_id,order_date,order_size,order_id
0,9,2016-08-01,3,0
1,9,2016-08-10,2,1
2,11,2016-07-02,1,2
3,24,2016-07-03,1,3
4,30,2016-08-13,2,4
...,...,...,...,...
19378,43796,2016-08-20,2,19378
19379,43797,2016-08-20,7,19379
19380,43798,2016-08-20,3,19380
19381,43799,2016-08-20,2,19381


In [35]:
df = pd.merge(df,order_id_generator,on=['user_id', 'order_date'], how='left')
df.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.9,30822,Mrs,...,2016-06-23,0,"(40, 50]",-1,49.9,0,1,1,1,9485
1,763,2016-06-24,2016-06-26,643,38,navy,30,49.9,29306,Mrs,...,2016-06-17,0,"(40, 50]",7,49.9,0,1,1,1,8978
2,3,2016-06-22,2016-06-27,270,xxl,grey,49,79.9,30823,Mrs,...,2015-03-15,1,"(40, 50]",465,79.9,0,1,1,9,9487
3,522,2016-06-23,2016-06-26,270,xxl,grey,49,79.9,11260,Mrs,...,2015-04-24,0,"(60, 70]",426,79.9,0,1,1,9,3653
4,25884,2016-07-03,2016-07-04,270,xxl,grey,49,79.9,1283,Mrs,...,2015-09-15,1,"(40, 50]",292,79.9,0,1,1,3,455


In [36]:
df_user_agg = df.groupby('user_id', as_index = False).size()
df_user_agg
df_user_agg[df_user_agg['size'] > 1]
df_reduced = df[df.user_id < 100].sort_values('order_date')
df_reduced.groupby('user_id').apply(display)

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
56373,47259,2016-08-01,2016-08-02,2081,40,black,40,219.9,9,Mrs,...,2016-03-02,1,"(40, 50]",152,219.9,0,1,1,3,0
56376,47261,2016-08-01,2016-08-24,1613,40,ancient,1,199.9,9,Mrs,...,2016-03-02,1,"(40, 50]",152,199.9,0,4,3,3,0
56375,47260,2016-08-01,2016-08-02,1922,40,grey,26,159.9,9,Mrs,...,2016-03-02,1,"(40, 50]",152,159.9,0,3,2,3,0
59507,62930,2016-08-10,2016-08-14,1613,40,black,1,199.9,9,Mrs,...,2016-03-02,0,"(40, 50]",161,199.9,0,5,2,2,1
56374,62931,2016-08-10,2016-08-14,2081,40,black,40,219.9,9,Mrs,...,2016-03-02,1,"(40, 50]",161,219.9,0,2,1,2,1


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
36400,24204,2016-07-02,2016-07-03,966,37,turquoise,69,44.9,11,Mrs,...,2015-02-17,0,"(40, 50]",501,44.9,0,1,1,1,2


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
17280,26128,2016-07-03,2016-07-04,103,41,grey,17,69.9,24,Mrs,...,2016-03-04,0,"(50, 60]",121,99.9,1,1,1,1,3


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
59091,67311,2016-08-13,2016-08-15,1658,l,black,90,189.9,30,Mrs,...,2016-01-22,1,"(60, 70]",204,189.9,0,2,2,2,4
55521,67312,2016-08-13,2016-08-15,1658,m,black,90,189.9,30,Mrs,...,2016-01-22,0,"(60, 70]",204,189.9,0,1,1,2,4


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
51098,65060,2016-08-11,2016-08-14,1617,40,green,6,69.9,40,Mrs,...,2015-02-17,0,"(50, 60]",541,69.9,0,1,1,2,5
61577,65059,2016-08-11,2016-08-14,1559,42,black,1,159.9,40,Mrs,...,2015-02-17,0,"(50, 60]",541,159.9,0,2,2,2,5


...

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
29548,21201,2016-07-01,2016-07-23,557,38,pink,1,64.9,90,Mrs,...,2015-02-17,0,"(50, 60]",500,64.9,0,1,1,1,18


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
11030,12572,2016-06-28,2016-07-05,860,l,ocher,42,49.9,94,Mrs,...,2016-02-29,0,"(50, 60]",120,99.9,1,3,1,4,19
28906,12569,2016-06-28,2016-07-02,334,40,grey,72,49.9,94,Mrs,...,2016-02-29,0,"(50, 60]",120,49.9,0,5,2,4,19
28909,12570,2016-06-28,2016-07-02,415,41,green,1,44.9,94,Mrs,...,2016-02-29,1,"(50, 60]",120,69.9,1,6,3,4,19
28913,12571,2016-06-28,2016-07-02,119,41,blue,43,44.9,94,Mrs,...,2016-02-29,1,"(50, 60]",120,59.9,1,7,4,4,19
34733,20389,2016-06-30,2016-07-24,58,l,green,31,44.9,94,Mrs,...,2016-02-29,0,"(50, 60]",122,44.9,0,8,4,4,20
25131,20392,2016-06-30,2016-07-27,370,l,ocher,31,44.9,94,Mrs,...,2016-02-29,0,"(50, 60]",122,49.9,1,4,3,4,20
5440,20390,2016-06-30,2016-07-24,32,m,red,3,12.4,94,Mrs,...,2016-02-29,0,"(50, 60]",122,24.9,1,2,2,4,20
4002,20391,2016-06-30,2016-07-24,434,l,green,28,49.9,94,Mrs,...,2016-02-29,0,"(50, 60]",122,59.9,1,1,1,4,20
42696,38670,2016-07-20,2016-07-23,1630,l,petrol,37,49.9,94,Mrs,...,2016-02-29,1,"(50, 60]",142,49.9,0,15,7,8,21
42671,38666,2016-07-20,2016-07-23,1628,l,petrol,3,24.9,94,Mrs,...,2016-02-29,0,"(50, 60]",142,24.9,0,11,3,8,21


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
30117,33973,2016-07-13,2016-07-16,367,43,blue,18,99.90,98,Mrs,...,2015-02-17,1,"(10, 20]",512,99.90,0,1,1,5,22
36364,33970,2016-07-13,2016-07-16,1333,33,purple,69,42.90,98,Mrs,...,2015-02-17,1,"(10, 20]",512,42.90,0,3,3,5,22
36712,33971,2016-07-13,2016-07-16,1373,34,red,69,39.90,98,Mrs,...,2015-02-17,0,"(10, 20]",512,39.90,0,4,4,5,22
38348,33974,2016-07-13,2016-07-16,375,44,brown,13,49.90,98,Mrs,...,2015-02-17,0,"(10, 20]",512,49.90,0,5,5,5,22
30570,33972,2016-07-13,2016-07-16,1332,35,red,69,44.90,98,Mrs,...,2015-02-17,1,"(10, 20]",512,44.90,0,2,2,5,22
65130,67269,2016-08-13,2016-08-20,2024,29,blue,51,69.90,98,Mrs,...,2015-02-17,1,"(10, 20]",543,69.90,0,18,13,15,23
65129,67268,2016-08-13,2016-08-20,2024,28,blue,51,69.90,98,Mrs,...,2015-02-17,1,"(10, 20]",543,69.90,0,17,12,15,23
65126,67264,2016-08-13,2016-08-20,2187,30,purple,51,79.90,98,Mrs,...,2015-02-17,1,"(10, 20]",543,79.90,0,15,10,15,23
61605,67263,2016-08-13,2016-08-14,2056,3,brown,21,69.95,98,Mrs,...,2015-02-17,1,"(10, 20]",543,69.95,0,8,3,15,23
65125,67262,2016-08-13,2016-08-14,2056,2,brown,21,69.95,98,Mrs,...,2015-02-17,1,"(10, 20]",543,69.95,0,14,9,15,23


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_reg_date,return,age,acc_age,maxprice,markdown,nth_order_of_user,nth_order_of_user_v2,order_size,order_id
14038,2837,2016-06-25,2016-06-26,502,10,green,21,45.90,99,Mrs,...,2016-03-31,0,"(40, 50]",86,45.90,0,4,3,3,24
14019,2836,2016-06-25,2016-06-26,128,l,ash,49,49.90,99,Mrs,...,2016-03-31,0,"(40, 50]",86,49.90,0,3,2,3,24
12288,2838,2016-06-25,2016-07-18,171,xl,black,3,24.90,99,Mrs,...,2016-03-31,0,"(40, 50]",86,24.90,0,2,1,3,24
23135,8999,2016-06-27,2016-07-02,128,l,grey,49,39.90,99,Mrs,...,2016-03-31,0,"(40, 50]",88,79.90,1,6,3,5,25
8300,9001,2016-06-27,2016-07-02,465,l,ash,49,29.90,99,Mrs,...,2016-03-31,0,"(40, 50]",88,59.90,1,1,1,5,25
25818,8998,2016-06-27,2016-07-02,263,m,green,5,34.90,99,Mrs,...,2016-03-31,0,"(40, 50]",88,39.90,1,7,4,5,25
25823,9000,2016-06-27,2016-07-02,128,m,ash,49,39.90,99,Mrs,...,2016-03-31,0,"(40, 50]",88,39.90,0,8,5,5,25
18567,9002,2016-06-27,2016-07-02,465,l,grey,49,29.90,99,Mrs,...,2016-03-31,0,"(40, 50]",88,59.90,1,5,2,5,25
56398,57348,2016-08-07,2016-08-09,2071,unsized,red,48,16.95,99,Mrs,...,2016-03-31,0,"(40, 50]",129,16.95,0,10,2,5,26
57924,57351,2016-08-07,2016-09-17,1802,xxl,navy,30,32.95,99,Mrs,...,2016-03-31,0,"(40, 50]",129,32.95,0,11,3,5,26


""
